Hal class definition

In [6]:
from highly_adaptive_lasso import HAL
from highly_adaptive_ridge import HAR


from sklearn.linear_model import Lasso, LassoCV, RidgeCV
import numpy as np
from data_generator import DataGenerator
from sklearn.model_selection import train_test_split

# cannot resolve in pipenv - BUT WHY?!? using base for now
import pandas as pd 
import matplotlib.pyplot as plt

Let's generate some data!

In [7]:
num_samples = 1000

# Generate synthetic data using the DataGenerator class (from Alejandro)
X, Y = DataGenerator.generate_data(num_samples)

# Split the data into training and testing sets
test_size = 0.9 # use 90% of the data for training
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

In [6]:

# # this displays the alpha values that were used in the LassoCV
# # NOTE: alpha refers to the regularization parameter (?)
# # TODO: Ask Alejandro about this
# alphas = model.lasso.alphas_
# alpha = model.lasso.alpha_

Training and evaluation

In [8]:
from sklearn.metrics import mean_squared_error
import time

# Initialize models
hal = HAL() 
har = HAR()  

# Fit models and record computation time
start_time = time.time()
hal.fit(X_train, y_train)
hal_time = time.time() - start_time

start_time = time.time()
har.fit(X_train, y_train)
har_time = time.time() - start_time

# Evaluate models
hal_mse = mean_squared_error(y_test, hal.predict(X_test))
har_mse = mean_squared_error(y_test, har.predict(X_test))

In [10]:
# Display results
print("HAL MSE: ", hal_mse)
print("HAR MSE: ", har_mse)
print("HAL time: ", hal_time)
print("HAR time: ", har_time)


HAL MSE:  1.3928847267876285
HAR MSE:  1.1853674246535966
HAL time:  42.82060885429382
HAR time:  0.04938101768493652
